In [ ]:
import pandas as  pd

import numpy as np

import matplotlib.pyplot as plt

import botocore


In [ ]:
# UPLOADING DATA
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_fa30d206090f4c45beb28cb5540c7194 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='sLkurfQzqxAX3kOp7nWKELBH-sC66GkHDJh0CEI7zGvw',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_fa30d206090f4c45beb28cb5540c7194.get_object(Bucket='machinelearning-donotdelete-pr-vtlqvcyuvm9np8',Key='data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

ds = pd.read_csv(body)
ds.head()



In [ ]:
ds.isnull().any()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb=LabelEncoder()

ds=ds.iloc[:,:].values



In [ ]:
ds[:,3]=lb.fit_transform(ds[:,3])

ds[:,7]=lb.fit_transform(ds[:,7])

In [ ]:
da=pd.DataFrame(ds)

In [ ]:
da.isnull().any()

In [ ]:
y=ds[:,7]

y=y.astype("int")

In [ ]:

da.drop(columns=7,inplace=True)


In [ ]:
x=da.iloc[:,:].values

x

In [ ]:
# TRAIN TEST SPLIT

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


In [ ]:
# FEATURE SCALING

from sklearn.preprocessing import MinMaxScaler

sc=MinMaxScaler()



In [ ]:
x_train



In [ ]:
x_train


# decision tree

# training

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt=DecisionTreeClassifier(criterion='entropy')

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scalar',sc),('dt',dt)])

In [ ]:
dt= pipeline.fit(x_train,y_train)

# predicting

In [ ]:
y_pred_dt=dt.predict(x_test)

# accuracy

In [ ]:
import sklearn.metrics as metrics

In [ ]:
fpr,tpr,threshold=metrics.roc_curve(y_test,y_pred_dt)

In [ ]:
roc_auc_DT=metrics.auc(fpr,tpr)

In [ ]:
roc_auc_DT

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred_dt)

In [ ]:
plt.plot(fpr,tpr,label='AUC = %0.2f' % roc_auc_DT)
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.title("roc_curve")
plt.legend()

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_credentials={
"access_key": "RWVUf6PMhFtWBgy86pjqji590_jahBPb8X-KIQ7uNEQp",
  "instance_id": "5571fc48-0a00-4138-804f-26a0a599bedb",
  "password": "93122f4a-4d26-4e73-9f66-27ec2987ebf6",
  "url": "https://eu-gb.ml.cloud.ibm.com",
  "username": "ed8619de-5beb-466a-8d45-98396a15e3a4"
}

In [ ]:
client=WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
model_props={client.repository.ModelMetaNames.AUTHOR_NAME:"Pranav",
            client.repository.ModelMetaNames.AUTHOR_EMAIL:"Pranav.mamtaji2019@vitbhopal.ac.in",
            client.repository.ModelMetaNames.NAME:"3dprinter"}

In [ ]:
client.repository.list()

In [ ]:
model_s=client.repository.store_model(dt,meta_props=model_props)

In [ ]:
published_model_uid=client.repository.get_model_uid(model_s)

In [ ]:
published_model_uid

In [ ]:
d=client.deployments.create(published_model_uid,name="3dprinter")

In [ ]:
scoring_endpoint=client.deployments.get_scoring_url(d)

In [ ]:
scoring_endpoint

In [ ]:
client.deployments.list()

In [ ]:
client.deployments.delete('7f772379-38b9-4ba9-9fea-f2078ab71b6c')

In [ ]:
dt.predict([[0.02,8,90,0,220,60,40,0,25,18,1.2]])